In [1]:
from ultralytics import YOLO
import cv2
import easyocr
import csv
import numpy as np
import string
from collections import defaultdict, Counter

In [2]:
# Inicializa el modelo YOLO para detectar vehículos y personas y el modelo de detección de matrículas
detector = YOLO("yolo11n.pt")  # Modelo YOLO para detectar vehículos y personas
detector_matriculas = YOLO("license_plate_detector.pt")  # Modelo para detectar matrículas

In [3]:
# Inicializa el lector de OCR
ocr_reader = easyocr.Reader(['en'])  # Cambia 'en' si necesitas otro idioma

# Diccionario para almacenar el historial de seguimiento
track_history = defaultdict(lambda: [])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
# Nombres de clases para los vehículos y personas
classNames = ["person", "bicycle", "car", "motorcycle", "bus", "truck"]
color_palette = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

dict_char_to_int = {
    'O': '0', 'I': '1', 'J': '3', 'A': '4', 'G': '6', 'S': '5'
}

dict_int_to_char = {
    '0': 'O', '1': 'I', '3': 'J', '4': 'A', '6': 'G', '5': 'S'
}

In [5]:
# Crear y abrir el archivo CSV para almacenar datos
with open("detecciones.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["fotograma", "tipo_objeto", "confianza", "identificador_tracking", 
                     "x1", "y1", "x2", "y2", "matricula_en_su_caso", "confianza_matricula", 
                     "mx1", "my1", "mx2", "my2", "texto_matricula"])


In [6]:
def license_complies_format(text):
    if len(text) != 7:
        return False

    if (text[0] in '0123456789' or text[2] in dict_char_to_int.keys()) and \
       (text[1] in '0123456789' or text[2] in dict_char_to_int.keys()) and \
       (text[2] in '0123456789' or text[2] in dict_char_to_int.keys()) and \
       (text[3] in '0123456789' or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    return False

In [7]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in range(7):
        license_plate_ += mapping[j].get(text[j], text[j])

    return license_plate_

In [8]:
def read_license_plate(license_plate_crop):
    detections = ocr_reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection
        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None

In [9]:
def procesar_y_anonimizar(imagen, frame_id):
    resultados = detector.track(imagen, persist=True, classes=[0, 2, 3, 5, 7])
    contador_clases = Counter()
    imagen_con_recuadros = imagen.copy()
    imagen_anonimizada = imagen.copy()

    for r in resultados:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confianza = round(float(box.conf[0]), 2)
            track_id = int(box.id[0].tolist()) if box.id is not None else -1
            cls = int(box.cls[0])

            if cls < 0 or cls >= len(classNames):
                print(f"Advertencia: Clase {cls} fuera de rango para classNames.")
                continue

            contador_clases[classNames[cls]] += 1

            # Dibuja el cuadro en el video sin anonimización
            color = color_palette[cls % len(color_palette)]
            cv2.rectangle(imagen_con_recuadros, (x1, y1), (x2, y2), color, 2)
            cv2.putText(imagen_con_recuadros, f"ID {track_id} {classNames[cls]}", 
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Aplica el desenfoque en el área detectada para el video anonimizado
            roi = imagen_anonimizada[y1:y2, x1:x2]
            blurred_roi = cv2.GaussianBlur(roi, (51, 51), 0)
            imagen_anonimizada[y1:y2, x1:x2] = blurred_roi

            # Si el objeto detectado es un vehículo, buscar matrícula
            if cls in [2, 3, 5, 7]:
                roi_vehiculo = imagen[y1:y2, x1:x2]
                resultados_matriculas = detector_matriculas(roi_vehiculo)

                abs_mx1, abs_my1, abs_mx2, abs_my2 = 0, 0, 0, 0
                texto_matricula = ""
                confianza_matricula = 0

                for m in resultados_matriculas[0].boxes:
                    mx1, my1, mx2, my2 = map(int, m.xyxy[0])
                    confianza_matricula = round(float(m.conf[0]), 2)

                    abs_mx1, abs_my1 = x1 + mx1, y1 + my1
                    abs_mx2, abs_my2 = x1 + mx2, y1 + my2

                    roi_matricula = imagen[abs_my1:abs_my2, abs_mx1:abs_mx2]
                    if roi_matricula.size > 0:
                        texto_matricula, confianza_matricula = read_license_plate(roi_matricula)

                    cv2.rectangle(imagen_con_recuadros, (abs_mx1, abs_my1), (abs_mx2, abs_my2), (0, 255, 255), 2)
                    if texto_matricula:
                        cv2.putText(imagen_con_recuadros, texto_matricula, 
                                    (abs_mx1, abs_my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    break  # Solo considera la primera matrícula detectada en el ROI

                with open("detecciones.csv", mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([frame_id, classNames[cls], confianza, track_id, x1, y1, x2, y2,
                                     1 if texto_matricula else 0, confianza_matricula, 
                                     abs_mx1, abs_my1, abs_mx2, abs_my2, texto_matricula])

    # Dibuja el conteo de cada clase en la esquina superior derecha en ambos videos
    y_offset = 20
    for cls_name, count in contador_clases.items():
        cv2.putText(imagen_con_recuadros, f"{cls_name}: {count}", (10, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(imagen_anonimizada, f"{cls_name}: {count}", (10, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 20

    return imagen_con_recuadros, imagen_anonimizada

In [10]:
video_capture = cv2.VideoCapture("sample3m.mp4")
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_con_recuadros = cv2.VideoWriter("output_con_recuadros.mp4", fourcc, fps, (width, height))
output_anonimizado = cv2.VideoWriter("output_anonimizado.mp4", fourcc, fps, (width, height))

frame_id = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    frame_id += 1
    imagen_con_recuadros, imagen_anonimizada = procesar_y_anonimizar(frame, frame_id)

    output_con_recuadros.write(imagen_con_recuadros)
    output_anonimizado.write(imagen_anonimizada)




# Liberar recursos
video_capture.release()
output_con_recuadros.release()
output_anonimizado.release()


0: 384x640 2 persons, 3 cars, 238.0ms
Speed: 28.6ms preprocess, 238.0ms inference, 16.4ms postprocess per image at shape (1, 3, 384, 640)

0: 640x256 (no detections), 100.2ms
Speed: 16.6ms preprocess, 100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 256)

0: 480x640 2 license_plates, 130.0ms
Speed: 10.2ms preprocess, 130.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 (no detections), 156.3ms
Speed: 0.0ms preprocess, 156.3ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 2 persons, 2 cars, 114.2ms
Speed: 4.0ms preprocess, 114.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 license_plate, 116.7ms
Speed: 7.7ms preprocess, 116.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 (no detections), 126.1ms
Speed: 0.0ms preprocess, 126.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 cars, 115.4ms
Speed: 